# 🏂 Holt Winters Model (Seasonal Version)
Now, it has the WINTER'S component, meaning *the season of winter will be covered* 😁 <br>
**Nah**, here winters is not the ***season***!

### 🤷‍♂️ But why Winters?
Now obviously you will get this doubt that *there can be a name like **holt** but, what about **winters** part? Does that show some kind of seasonality*? Hah, looking on the internet:

> Holt-Winter's Exponential Smoothing as named after its **two contributors**: Charles ***Holt*** and Peter ***Winter's*** is one of the oldest time series analysis techniques which takes into account the trend and seasonality while doing the forecasting.

અરે યાર, winters કોઈનો નામ પણ હોય! 🙄